## 1. Import Required Libraries

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Time series tools
from datetime import datetime, timedelta

# Statistical analysis
from scipy import stats

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 2. Load Historical Sales Data

**Instructions**: Place your dataset in the `data/raw/` folder and update the file path below.

In [ ]:
# Load data from CSV
# Adjust the file path and column names based on your dataset
try:
    df = pd.read_csv('../data/raw/sales_data.csv')
    print(f"✓ Data loaded successfully: {df.shape[0]} rows, {df.shape[1]} columns")
except FileNotFoundError:
    print("⚠ Data file not found. Please ensure your CSV is in data/raw/ folder.")
    print("\nSupported datasets:")
    print("- Superstore Sales Dataset (Kaggle)")
    print("- Retail Sales Forecasting (Kaggle)")
    print("- Rossmann Store Sales (Kaggle)")

## 3. Data Structure & Overview

In [ ]:
# Display basic information
print("Data Shape:", df.shape)
print("\nFirst 5 Rows:")
print(df.head())
print("\nData Types:")
print(df.dtypes)
print("\nBasic Statistics:")
print(df.describe())

In [ ]:
# Check for missing values
print("Missing Values:")
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing_Count': missing.values,
    'Missing_Percent': missing_percent.values
})
print(missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False))

## 4. Data Cleaning & Preprocessing

In [ ]:
# Create a copy for processing
df_clean = df.copy()

# Identify date column (adjust based on your dataset)
# Common column names: 'Date', 'OrderDate', 'TransactionDate', 'DateTime'
date_columns = [col for col in df_clean.columns if 'date' in col.lower() or 'time' in col.lower()]
print(f"Potential date columns: {date_columns}")

# Convert date column to datetime
if date_columns:
    date_col = date_columns[0]
    df_clean[date_col] = pd.to_datetime(df_clean[date_col], errors='coerce')
    print(f"✓ Converted '{date_col}' to datetime")

In [ ]:
# Remove duplicates
duplicates_before = len(df_clean)
df_clean = df_clean.drop_duplicates()
duplicates_removed = duplicates_before - len(df_clean)
print(f"✓ Removed {duplicates_removed} duplicate rows")

# Handle missing values in sales column
sales_columns = [col for col in df_clean.columns if 'sales' in col.lower() or 'amount' in col.lower() or 'revenue' in col.lower()]
print(f"\nPotential sales columns: {sales_columns}")

In [ ]:
# Fill missing values strategy
# For numerical columns: forward fill or mean
numerical_cols = df_clean.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col].fillna(df_clean[col].mean(), inplace=True)
        print(f"✓ Filled missing values in '{col}' with mean")

# For categorical columns: forward fill or mode
categorical_cols = df_clean.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)
        print(f"✓ Filled missing values in '{col}' with mode")

In [ ]:
# Verify cleaning
print("\nCleaning Summary:")
print(f"Total rows: {len(df_clean)}")
print(f"Missing values: {df_clean.isnull().sum().sum()}")
print(f"\nData types after cleaning:")
print(df_clean.dtypes)

## 5. Time Series Data Preparation

In [ ]:
# Identify and set date column
if date_columns:
    date_col = date_columns[0]
    sales_col = sales_columns[0] if sales_columns else df_clean.columns[2]
    
    # Create daily sales aggregation for time series
    daily_sales = df_clean.groupby(date_col)[sales_col].sum().reset_index()
    daily_sales.columns = ['date', 'sales']
    daily_sales = daily_sales.sort_values('date')
    daily_sales.set_index('date', inplace=True)
    
    print(f"Daily sales data shape: {daily_sales.shape}")
    print(f"Date range: {daily_sales.index.min()} to {daily_sales.index.max()}")
    print(f"\nFirst 10 days:")
    print(daily_sales.head(10))

## 6. Exploratory Data Analysis (EDA)

In [ ]:
# Overall sales trends
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Time series plot
axes[0, 0].plot(daily_sales.index, daily_sales['sales'], linewidth=1.5, color='steelblue')
axes[0, 0].set_title('Daily Sales Over Time', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Sales')
axes[0, 0].grid(True, alpha=0.3)

# Distribution of sales
axes[0, 1].hist(daily_sales['sales'], bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Distribution of Daily Sales', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Sales Amount')
axes[0, 1].set_ylabel('Frequency')

# Box plot for outlier detection
axes[1, 0].boxplot(daily_sales['sales'])
axes[1, 0].set_title('Box Plot of Sales (Outlier Detection)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Sales')

# Monthly aggregation
if date_col:
    monthly_sales = df_clean.set_index(date_col)[sales_col].resample('M').sum()
    axes[1, 1].bar(range(len(monthly_sales)), monthly_sales.values, color='lightgreen', edgecolor='black')
    axes[1, 1].set_title('Monthly Sales Aggregation', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Month')
    axes[1, 1].set_ylabel('Total Sales')

plt.tight_layout()
plt.savefig('../outputs/01_sales_overview.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: 01_sales_overview.png")

In [ ]:
# Statistical summary
print("\n=== SALES STATISTICS ===")
print(f"Total Sales: ${daily_sales['sales'].sum():,.2f}")
print(f"Average Daily Sales: ${daily_sales['sales'].mean():,.2f}")
print(f"Median Daily Sales: ${daily_sales['sales'].median():,.2f}")
print(f"Standard Deviation: ${daily_sales['sales'].std():,.2f}")
print(f"Min Sales: ${daily_sales['sales'].min():,.2f}")
print(f"Max Sales: ${daily_sales['sales'].max():,.2f}")
print(f"Coefficient of Variation: {(daily_sales['sales'].std() / daily_sales['sales'].mean()):.2%}")

## 7. Seasonality & Trend Analysis

In [ ]:
# Extract time components for analysis
daily_sales['year'] = daily_sales.index.year
daily_sales['month'] = daily_sales.index.month
daily_sales['quarter'] = daily_sales.index.quarter
daily_sales['day_of_week'] = daily_sales.index.dayofweek
daily_sales['week_of_year'] = daily_sales.index.isocalendar().week

print("✓ Added time components for seasonality analysis")

# Monthly patterns
monthly_pattern = daily_sales.groupby('month')['sales'].agg(['mean', 'std', 'count'])
monthly_pattern.index = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
print("\nMonthly Sales Pattern:")
print(monthly_pattern)

In [ ]:
# Visualize seasonality
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# By month
daily_sales.groupby('month')['sales'].mean().plot(ax=axes[0, 0], marker='o', color='steelblue', linewidth=2)
axes[0, 0].set_title('Average Sales by Month', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Month')
axes[0, 0].set_ylabel('Average Sales')
axes[0, 0].grid(True, alpha=0.3)

# By day of week
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_avg = daily_sales.groupby('day_of_week')['sales'].mean()
axes[0, 1].bar(range(len(day_avg)), day_avg.values, color='coral', edgecolor='black')
axes[0, 1].set_xticks(range(7))
axes[0, 1].set_xticklabels(day_names, rotation=45)
axes[0, 1].set_title('Average Sales by Day of Week', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Average Sales')

# By quarter
daily_sales.groupby('quarter')['sales'].mean().plot(ax=axes[1, 0], marker='s', color='lightgreen', linewidth=2)
axes[1, 0].set_title('Average Sales by Quarter', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Quarter')
axes[1, 0].set_ylabel('Average Sales')
axes[1, 0].grid(True, alpha=0.3)

# Year-over-year if multiple years
if len(daily_sales['year'].unique()) > 1:
    for year in sorted(daily_sales['year'].unique()):
        year_data = daily_sales[daily_sales['year'] == year].groupby('month')['sales'].mean()
        axes[1, 1].plot(year_data.index, year_data.values, marker='o', label=str(year), linewidth=2)
    axes[1, 1].set_title('Year-over-Year Monthly Trends', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Month')
    axes[1, 1].set_ylabel('Average Sales')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../outputs/02_seasonality_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: 02_seasonality_analysis.png")

## 8. Key Findings & Insights

In [ ]:
print("\n=== KEY FINDINGS ===")
print(f"\n1. Data Period: {daily_sales.index.min().date()} to {daily_sales.index.max().date()}")
print(f"   Total Days: {len(daily_sales)} days")

# Trend direction
first_half_avg = daily_sales['sales'].iloc[:len(daily_sales)//2].mean()
second_half_avg = daily_sales['sales'].iloc[len(daily_sales)//2:].mean()
trend_change = ((second_half_avg - first_half_avg) / first_half_avg) * 100
print(f"\n2. Trend: {'Upward' if trend_change > 0 else 'Downward'} ({trend_change:+.1f}%)")

# Seasonality strength
monthly_std = daily_sales.groupby('month')['sales'].mean().std()
monthly_mean = daily_sales.groupby('month')['sales'].mean().mean()
seasonality_strength = (monthly_std / monthly_mean) * 100
print(f"\n3. Seasonality Strength: {seasonality_strength:.1f}%")
print("   (Higher values indicate stronger seasonal patterns)")

# Peak and low seasons
peak_month = daily_sales.groupby('month')['sales'].mean().idxmax()
low_month = daily_sales.groupby('month')['sales'].mean().idxmin()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
print(f"\n4. Peak Season: {month_names[peak_month-1]} | Low Season: {month_names[low_month-1]}")

# Day of week patterns
peak_day = daily_sales.groupby('day_of_week')['sales'].mean().idxmax()
print(f"\n5. Busiest Day: {day_names[peak_day]}")

# Volatility
daily_returns = daily_sales['sales'].pct_change().dropna()
volatility = daily_returns.std()
print(f"\n6. Daily Sales Volatility: {volatility:.2%}")
print("\n✓ Data exploration complete! Ready for feature engineering.")